In [1]:
import torch
import torchvision 
import torch.nn as nn
import torchvision.models as models
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split
torch.manual_seed(42)
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
import copy

In [2]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

train_dataset, val_dataset = random_split(train_dataset, [45000, 5000])

batch_size = 1000

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('NO GPU AVAILABLE ERROR')
    device = torch.device("cpu")
    
train_loader = DataLoader(train_dataset,
                        batch_size=batch_size,
                        shuffle=True)

val_loader = DataLoader(val_dataset,
                        batch_size=batch_size,
                        shuffle=True)

test_loader = DataLoader(test_dataset,
                        batch_size=batch_size,
                        shuffle=False)

Files already downloaded and verified
Files already downloaded and verified
There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
alexnet = torchvision.models.alexnet(pretrained = False)
pretrained_alexnet = torchvision.models.alexnet(pretrained = True)

model = nn.Sequential(
    torchvision.transforms.Resize((63,63)),
    alexnet,
    nn.ReLU(),
    nn.Linear(1000, 10)
)

finetune_model = nn.Sequential(
    torchvision.transforms.Resize((63,63)),
    pretrained_alexnet,
    nn.ReLU(),
    nn.Linear(1000, 10)
)

featureextract_model = copy.deepcopy(finetune_model)

# freeze feature extraction (13 layers)
for i, param in enumerate(featureextract_model.parameters()):
    if i < 13:
        param.requires_grad = False

In [4]:
def train(epochs, optimizer, loss_fn, train_loader, val_loader, model, name):
    for epoch in range(epochs):
        model.train()
        for i, (data, labels) in enumerate(train_loader):
            data = data.to(device)
            labels = labels.to(device)

            pred = model(data)
            loss = loss_fn(pred, labels)
            loss.backward()
            optimizer.step()
            writer.add_scalar(f"{name} Loss/train", loss, epoch)
            optimizer.zero_grad()
            print(
                f'\rEpoch {epoch+1} [{i+1}/{len(train_loader)}] - Loss: {loss}',
                end=''
            )

        print('\n*************************************')
        print('Validation the model after epoch:', epoch)
        accuracy(model, val_loader, epoch, f"{name} Accuracy/validation")

def accuracy(model, data_loader, epoch, name):
    correct, total = 0, 0
    with torch.no_grad():
        model.eval()
        for _, (data, labels) in enumerate(data_loader):
            data = data.to(device)
            labels = labels.to(device)
            
            pred = model(data)
            for i in range(len(labels)):
                pr = torch.argmax(pred[i], dim=-1)
                if pr == labels[i]:
                    correct += 1
                total += 1
        print(correct, total, correct/total)
        writer.add_scalar(name, correct/total, epoch)

In [5]:
model.to(device)
finetune_model.to(device)
featureextract_model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
epochs = 20

In [6]:
print('Training AlexNet from scratch')
optimizer = torch.optim.Adam(model.parameters())
train(epochs, optimizer, loss_fn, train_loader, val_loader, model, "Alexnet")
print('Test Accuracy')
accuracy(model, test_loader, 0, "Alexnet Accuracy/test")

Training AlexNet from scratch
Epoch 1 [45/45] - Loss: 2.3019835948944094
*************************************
Validation the model after epoch: 0
512 5000 0.1024
Epoch 2 [45/45] - Loss: 1.9955395460128784
*************************************
Validation the model after epoch: 1
841 5000 0.1682
Epoch 3 [45/45] - Loss: 1.8838280439376836
*************************************
Validation the model after epoch: 2
1353 5000 0.2706
Epoch 4 [45/45] - Loss: 1.6800491809844978
*************************************
Validation the model after epoch: 3
1544 5000 0.3088
Epoch 5 [45/45] - Loss: 1.5578992366790771
*************************************
Validation the model after epoch: 4
2018 5000 0.4036
Epoch 6 [45/45] - Loss: 1.4129625558853153
*************************************
Validation the model after epoch: 5
2319 5000 0.4638
Epoch 7 [45/45] - Loss: 1.3021799325942993
*************************************
Validation the model after epoch: 6
2614 5000 0.5228
Epoch 8 [45/45] - Loss: 1.21552860

In [7]:
print('Training AlexNet using Fine-Tuning')
optimizer = torch.optim.Adam(finetune_model.parameters())
train(epochs, optimizer, loss_fn, train_loader, val_loader, finetune_model, "Alexnet Fine-Tuning")
print('Test Accuracy')
accuracy(finetune_model, test_loader, 0, "Alexnet Fine-Tuning Accuracy/test")

Training AlexNet using Fine-Tuning
Epoch 1 [45/45] - Loss: 2.0327751636505127
*************************************
Validation the model after epoch: 0
1159 5000 0.2318
Epoch 2 [45/45] - Loss: 1.6401226520538334
*************************************
Validation the model after epoch: 1
2019 5000 0.4038
Epoch 3 [45/45] - Loss: 1.3341715335845947
*************************************
Validation the model after epoch: 2
2561 5000 0.5122
Epoch 4 [45/45] - Loss: 1.2036085128784187
*************************************
Validation the model after epoch: 3
2822 5000 0.5644
Epoch 5 [45/45] - Loss: 0.9623289108276367
*************************************
Validation the model after epoch: 4
3225 5000 0.645
Epoch 6 [45/45] - Loss: 0.8623090982437134
*************************************
Validation the model after epoch: 5
3328 5000 0.6656
Epoch 7 [45/45] - Loss: 0.7087350487709045
*************************************
Validation the model after epoch: 6
3569 5000 0.7138
Epoch 8 [45/45] - Loss: 0.68

In [8]:
print('Training AlexNet using Feature Extraction')
optimizer = torch.optim.Adam(featureextract_model.parameters())
train(epochs, optimizer, loss_fn, train_loader, val_loader, featureextract_model, "Alexnet Feature Extraction")
print('Test Accuracy')
accuracy(featureextract_model, test_loader, 0, "Alexnet Feature Extraction Accuracy/test")

Training AlexNet using Feature Extraction
Epoch 1 [45/45] - Loss: 1.3447772264480594
*************************************
Validation the model after epoch: 0
2725 5000 0.545
Epoch 2 [45/45] - Loss: 1.2082532644271857
*************************************
Validation the model after epoch: 1
2846 5000 0.5692
Epoch 3 [45/45] - Loss: 1.1594625711441045
*************************************
Validation the model after epoch: 2
2914 5000 0.5828
Epoch 4 [45/45] - Loss: 1.1182472705841064
*************************************
Validation the model after epoch: 3
2962 5000 0.5924
Epoch 5 [45/45] - Loss: 1.0970376729965217
*************************************
Validation the model after epoch: 4
2956 5000 0.5912
Epoch 6 [45/45] - Loss: 1.0950981378555298
*************************************
Validation the model after epoch: 5
2959 5000 0.5918
Epoch 7 [45/45] - Loss: 1.0373317003250122
*************************************
Validation the model after epoch: 6
2973 5000 0.5946
Epoch 8 [45/45] - Los

In [9]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 16, 5, 1, 2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5, 1, 2)
        self.fc1 = nn.Linear(32 * 7 * 7, 10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x))) # 28x28 > 14x14
        x = self.pool(self.relu(self.conv2(x))) # 14x14 > 7x7
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x
    
# Create an instance of the model
model_mnist = Model()
model_mnist.to(device)

Model(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
  (relu): ReLU()
)

In [10]:
train_dataset_mnist = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=ToTensor())
train_dataset_mnist, val_dataset_mnist = random_split(train_dataset_mnist, [55000, 5000])
test_dataset_mnist = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=ToTensor())

batch_size = 1000
epochs = 10

train_loader_mnist = DataLoader(train_dataset_mnist,
                        batch_size=batch_size,
                        shuffle=True)
val_loader_mnist = DataLoader(val_dataset_mnist,
                        batch_size=batch_size,
                        shuffle=True)

test_loader_mnist = DataLoader(test_dataset_mnist,
                        batch_size=batch_size,
                        shuffle=False)

1.0%

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw


102.8%
0.1%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100.0%
112.7%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [11]:
print('Training CNN on MNIST dataset')
optimizer = torch.optim.Adam(model_mnist.parameters(), lr=0.001)
train(epochs, optimizer, loss_fn, train_loader_mnist, val_loader_mnist, model_mnist, "CNN MNIST")
print('Test Accuracy:')
accuracy(model_mnist, test_loader_mnist, 0, "CNN MNIST Accuracy/test")

Training CNN on MNIST dataset
Epoch 1 [55/55] - Loss: 0.29157838225364685
*************************************
Validation the model after epoch: 0
4555 5000 0.911
Epoch 2 [55/55] - Loss: 0.16047182679176334
*************************************
Validation the model after epoch: 1
4743 5000 0.9486
Epoch 3 [55/55] - Loss: 0.12898932397365574
*************************************
Validation the model after epoch: 2
4809 5000 0.9618
Epoch 4 [55/55] - Loss: 0.10024007409811028
*************************************
Validation the model after epoch: 3
4843 5000 0.9686
Epoch 5 [55/55] - Loss: 0.07718485593795776
*************************************
Validation the model after epoch: 4
4875 5000 0.975
Epoch 6 [55/55] - Loss: 0.049719043076038364
*************************************
Validation the model after epoch: 5
4883 5000 0.9766
Epoch 7 [55/55] - Loss: 0.082723706960678146
*************************************
Validation the model after epoch: 6
4899 5000 0.9798
Epoch 8 [55/55] - Loss: 0

In [12]:
model_svhn = Model()
model_svhn_finetune = copy.deepcopy(model_mnist)
model_svhn_featureextract = copy.deepcopy(model_mnist)

# freeze feature extraction (4 layers)
for i, param in enumerate(model_svhn_featureextract.parameters()):
    if i < 4:
        param.requires_grad = False

model_svhn.to(device)
model_svhn_finetune.to(device)
model_svhn_featureextract.to(device)

Model(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
  (relu): ReLU()
)

In [14]:
from torchvision import transforms
transforms = transforms.Compose([
    transforms.Resize((28,28)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

In [15]:
train_dataset_svhn = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transforms)
test_dataset_svhn = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transforms)

train_dataset_svhn, val_dataset_svhn = random_split(train_dataset_svhn, [73257-10000, 10000])

train_loader_svhn = DataLoader(train_dataset_svhn,
                        batch_size=batch_size,
                        shuffle=True)                        

val_loader_svhn = DataLoader(train_dataset_svhn,
                        batch_size=batch_size,
                        shuffle=True)                        


test_loader_svhn = DataLoader(test_dataset_svhn,
                        batch_size=batch_size,
                        shuffle=False)

100.0%


100.0%


In [16]:
print('Training CNN on SVHN dataset')
optimizer = torch.optim.Adam(model_svhn.parameters(), lr=0.001)
train(epochs, optimizer, loss_fn, train_loader_svhn, val_loader_svhn, model_svhn, "CNN SVHN")
print('Test Accuracy:')
accuracy(model_svhn, test_loader_svhn, 0, "CNN SVHN Accuracy/test")

Training CNN on SVHN dataset
Epoch 1 [64/64] - Loss: 2.2119908332824707
*************************************
Validation the model after epoch: 0
13724 63257 0.21695622618840602
Epoch 2 [64/64] - Loss: 1.3105114698410034
*************************************
Validation the model after epoch: 1
36999 63257 0.5848996948954266
Epoch 3 [64/64] - Loss: 0.9640682339668274
*************************************
Validation the model after epoch: 2
46267 63257 0.7314131242392147
Epoch 4 [64/64] - Loss: 0.6737480759620667
*************************************
Validation the model after epoch: 3
49536 63257 0.7830911993929526
Epoch 5 [64/64] - Loss: 0.8399789333343506
*************************************
Validation the model after epoch: 4
51161 63257 0.808780055962186
Epoch 6 [64/64] - Loss: 0.6650354862213135
*************************************
Validation the model after epoch: 5
52305 63257 0.8268650109869263
Epoch 7 [64/64] - Loss: 0.6380900740623474
*************************************
Va

In [17]:
print('Training CNN using Fine-Tuning from MNIST to SVHN dataset')
optimizer = torch.optim.Adam(model_svhn_finetune.parameters(), lr=0.001)
train(epochs, optimizer, loss_fn, train_loader_svhn, val_loader_svhn, model_svhn_finetune, "CNN SVHN Fine-Tuning")
print('Test Accuracy:')
accuracy(model_svhn_finetune, test_loader_svhn, 0, "CNN SVHN Fine-Tuning Accuracy/test")

Training CNN using Fine-Tuning from MNIST to SVHN dataset
Epoch 1 [64/64] - Loss: 1.5334813594818115
*************************************
Validation the model after epoch: 0
35194 63257 0.556365303444678
Epoch 2 [64/64] - Loss: 0.9767907261848451
*************************************
Validation the model after epoch: 1
45905 63257 0.7256904374219454
Epoch 3 [64/64] - Loss: 0.7508372068405151
*************************************
Validation the model after epoch: 2
48419 63257 0.7654330746004395
Epoch 4 [64/64] - Loss: 0.7203544974327087
*************************************
Validation the model after epoch: 3
50484 63257 0.7980776831022653
Epoch 5 [64/64] - Loss: 0.7950326204299927
*************************************
Validation the model after epoch: 4
51643 63257 0.8163997660337986
Epoch 6 [64/64] - Loss: 0.6251319050788879
*************************************
Validation the model after epoch: 5
51823 63257 0.8192453009153137
Epoch 7 [64/64] - Loss: 0.6101942658424377
************

In [26]:
print('Training CNN using Feature Extraction from MNIST to SVHN dataset')
optimizer = torch.optim.Adam(model_svhn_featureextract.parameters(), lr=0.001)
train(epochs, optimizer, loss_fn, train_loader_svhn, val_loader_svhn, model_svhn_featureextract, "CNN SVHN Feature Extraction")
print('Test Accuracy:')
accuracy(model_svhn_featureextract, test_loader_svhn, 0, "CNN SVHN Feature Extraction Accuracy/test")

Training CNN using Feature Extraction from MNIST to SVHN dataset
Epoch 1 [64/64] - Loss: 1.5267637968063354
*************************************
Validation the model after epoch: 0
31538 63257 0.4985693282956827
Epoch 2 [64/64] - Loss: 1.3291512727737427
*************************************
Validation the model after epoch: 1
38867 63257 0.6144300235547054
Epoch 3 [64/64] - Loss: 1.1845867633819587
*************************************
Validation the model after epoch: 2
43773 63257 0.6919866576031111
Epoch 4 [64/64] - Loss: 1.0370007753372192
*************************************
Validation the model after epoch: 3
45473 63257 0.7188611537063092
Epoch 5 [64/64] - Loss: 1.0482565164566045
*************************************
Validation the model after epoch: 4
46264 63257 0.7313656986578561
Epoch 6 [64/64] - Loss: 0.8969573974609375
*************************************
Validation the model after epoch: 5
47036 63257 0.7435698815941318
Epoch 7 [64/64] - Loss: 0.9260016083717346
****